# Scenario

A client approaches me to solve a machine learning problem for them. They run a pathology lab that processes histological images for healthcare providers and they have created a product that measures the same features as in the *Wisconsin breast cancer data set* though using different acquisitions and processing methods. This makes their method much faster than existing ones, but it is also slightly noisier. The dataset contains measurements of several features that describe characteristics of cell nuclei present in digital images of breast tissue samples. These features include the mean, standard error, and "worst" (i.e., largest) values of measurements such as radius, texture, perimeter, area, smoothness, compactness, concavity, concave points, symmetry, and fractal dimension. Each feature is represented by a numeric value, and each data point represents a sample from either a malignant (cancerous) or benign (non-cancerous) breast tissue.

The client want to be able to diagnose *malignant* cancer (and distinguish them from *benign* growths) by employing machine learning techniques, and they have asked to implement this for them.

Their requirements are:
 1) have at least a 95% probability of detecting malignant cancer when it is present;
 2) have no more than 1 in 10 healthy cases (those with benign tumours) labelled as positive (malignant).
 
They have hand-labelled 300 samples, which is all they have at the moment.

In [1]:
# Common imports
import sys
import numpy as np
import time
import pandas as pd
import sklearn
from sklearn import tree
from sklearn import svm
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
import re

# Plot setup
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
mpl.rc('axes', labelsize=7)
mpl.rc('xtick', labelsize=6)
mpl.rc('ytick', labelsize=6)
mpl.rc('figure', dpi=240)
plt.close('all')

Load the dataset

In [2]:
data = pd.read_csv("assignment2.csv")

feature_names = data.columns[1:].values

step1_data = data.copy()

In [51]:
step1_data

,label,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,malignant,15.494654,15.902542,103.008265,776.437239,0.104239,0.168660,0.170572,0.085668,0.205053,...,19.522957,22.427276,135.128520,1286.903131,0.142725,0.407483,0.445992,0.171662,0.353211,0.097731
1,malignant,16.229871,18.785613,105.176755,874.712003,0.091843,0.092548,0.081681,0.053670,0.180435,...,19.140235,24.905156,123.886045,1234.499997,0.129135,0.223918,0.248846,0.136735,0.284427,0.085758
2,malignant,16.345671,20.114076,107.083804,872.563251,0.099924,0.123799,0.128788,0.078310,0.189756,...,19.144816,25.601433,125.113036,1202.749973,0.135017,0.314402,0.332505,0.161497,0.313038,0.084340
3,malignant,13.001009,19.876997,85.889775,541.281012,0.113423,0.173069,0.146214,0.069574,0.212078,...,15.565911,26.145119,102.958265,737.655082,0.161390,0.485912,0.430007,0.167254,0.432297,0.117705
4,malignant,16.416060,17.397533,107.857386,891.516818,0.097321,0.111530,0.125971,0.068575,0.179562,...,18.620376,22.306233,124.002529,1139.490971,0.133950,0.230996,0.316620,0.131715,0.269591,0.080497
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,benign,14.048464,17.186671,90.974271,637.474225,0.094969,0.091549,0.063532,0.039494,0.173324,...,15.790651,22.538529,103.423320,819.408970,0.126466,0.206701,0.192139,0.095350,0.287380,0.078520
296,benign,12.879033,16.767790,83.123369,539.225356,0.092146,0.083986,0.059347,0.035404,0.167690,...,14.358919,21.955513,93.620160,684.694077,0.118165,0.191978,0.180949,0.083989,0.263879,0.078279
297,malignant,13.123052,18.793057,84.897717,555.002209,0.098036,0.090178,0.066586,0.043711,0.172389,...,14.991646,24.820718,97.933068,726.695117,0.126203,0.201766,0.202433,0.100361,0.256863,0.079667
298,benign,14.411991,18.970674,93.423809,671.128126,0.086304,0.090118,0.070882,0.039482,0.175789,...,16.555187,25.591332,108.978466,893.818250,0.120338,0.246945,0.236415,0.105354,0.280900,0.081828


In [4]:
# Separate the mean, standard error, and worst groups into separate dataframes
mean_group = data.filter(like='mean')
error_group = data.filter(like='error')
worst_group = data.filter(like='worst')

data['label'] = data['label'].map({'malignant': 1, 'benign': 0})

corr = data.corr()["label"].sort_values(ascending=False)
#print(corr.head(4))
# ranked_features = ["Feature1 name", "Feature2 name", "Feature3 name"] 
ranked_features = ["worst concave points", "worst perimeter", "worst radius"] 

step2_data = ranked_features

In [5]:
# corr
# print(corr.head(4))
# data.iloc[:,0]

<!-- END QUESTION -->

Convert the labels into 0s and 1s so that benign is represented by 0 and malignant is represented by 1.

In [6]:
from sklearn.model_selection import train_test_split

# Convert labels to 0 (benign) and 1 (malignant)

# Split dataset into training, validation, and test sets

bigtrain_set, test_set = train_test_split(data, test_size=0.2, random_state=42, stratify=data.iloc[:,0])
train_set, val_set = train_test_split(bigtrain_set, test_size=0.25, random_state=42, stratify=bigtrain_set.iloc[:,0])

X_train = train_set.iloc[:,1:]
y_train = train_set.iloc[:,0]
X_test = test_set.iloc[:,1:]
y_test = test_set.iloc[:,0]
X_val = val_set.iloc[:,1:]
y_val = val_set.iloc[:,0]


# Print the size of each resulting subset
print("Training set size:", len(X_train))
print("Validation set size:", len(X_val))
print("Test set size:", len(X_test))

step3_data = [len(X_train) , len(X_val) , len(X_test)]

Training set size: 180
Validation set size: 60
Test set size: 60


In [7]:
# X_test

Baseline measurements 

For our classification task we will consider **three simple baseline cases**:
1) predicting all samples to be negative (class 1)
2) predicting all samples to be positive (class 2)
3) making a random prediction for each sample with equal probability for each class 

In [8]:
# create arrays to hold the predicted labels for each baseline case
baseline1_preds = np.full(data.iloc[:,0].shape,0)
baseline2_preds = np.full(data.iloc[:,0].shape,1)
baseline3_preds = np.random.randint(low=0, high=2, size=(data.iloc[:,0].shape))

step4_data = [baseline1_preds, baseline2_preds, baseline3_preds] 

In [9]:
# step4_data

In [10]:
...
...
from sklearn import metrics

def calculate_metrics(y_true, y_pred):   
    rec = metrics.recall_score(y_true, y_pred)
    prec = metrics.precision_score(y_true, y_pred)
    auc = metrics.roc_auc_score(y_true, y_pred)
    f1 = metrics.f1_score(y_true, y_pred)

    return rec, prec, auc, f1

step5_data = calculate_metrics

Create a function which takes in the true labels and the predictions and returns the following parameters:

* Number of True Positives (TP)
* Number of True Negatives (TN)
* Number of False Positive (FP)
* Number of False Negative (FN)

In [11]:
# This function returns the number of TP, TN, FP, FN.
def get_values(y_true, y_pred):
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    
    for i in range(len(y_true)):
        if y_true[i] == 1 and y_pred[i] == 1:
            tp = tp+1
            
        elif y_true[i] == 0 and y_pred[i] == 0:
            tn = tn+1
            
        elif y_true[i] == 0 and y_pred[i] == 1:
            fp = fp+1
            
        elif y_true[i] == 1 and y_pred[i] == 0:
            fn = fn+1
    return tp, tn, fp, fn

step6_data = get_values

Based on the above baseline tests and the client's requirements, **choose a F1 Score** to use for evaluating/driving my machine learning methods.

In [12]:
# Choose the best metric of the five and assign to the variable below. The variable takes a string.
chosen_metric = "f1"

step7_data = chosen_metric

**Creating a SGD baseline**

In [13]:
from sklearn.impute import SimpleImputer
from sklearn.linear_model import SGDClassifier
# Create pre-processing pipeline
preproc_pl = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Create SGD model pipeline which has preproc_pl as the pre-processing part
sgd_base = Pipeline([('preprocessing', preproc_pl), 
                     ('sgd', SGDClassifier(random_state=42))])

# fit the SGD pipeline on the training data
sgd_base.fit(X_train, y_train)
# evaluate on the validation data
y_pred = sgd_base.predict(X_val)

step8_data = sgd_base

In [14]:
# step8_data

On the validation data, calculate and print or display the **normalized** version of the confusion matrix.

In [15]:
from sklearn.metrics import confusion_matrix
# Put the confusion matrix here
cmat = confusion_matrix(y_val, y_pred, normalize='true')

# Display or print cmat
print("Normalized Confusion Matrix:\n", cmat)

# Assign the required probability to the variable below
required_prob = cmat[1, 0]
# Choose which of the client's two criteria does this relate to (1 or 2)
criteria = 1
# Does the baseline model satisfy this criterion? ("YES" or "NO")
satisfy = "NO" if calculate_metrics(y_val, y_pred)[3] < 0.95 else "YES"

step9_data = cmat, required_prob

Normalized Confusion Matrix:
 [[1.         0.        ]
 [0.10344828 0.89655172]]


In [16]:
# step9_data

### Main classifier 

**Train and optimise the hyperparameters** to give the best performance for a **KNN (K-Nearest Neighbour)** classifier. For this model consider and  evaluate the following parameters:

* n_neighbors (with no value greater than 20)
* weights
* algorithm
* metric
* p (with no value greater than 20)

In [17]:

from sklearn.neighbors import KNeighborsClassifier

# Put the pipeline with the appropriate model 

knn_pl = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
    ('knn', KNeighborsClassifier())
])

parameters_knn = {
    'knn__n_neighbors': [1,5,10,15,20],
    'knn__weights': ['uniform', 'distance'],
    'knn__algorithm': ['ball_tree', 'kd_tree', 'brute'],
    'knn__metric': ['euclidean', 'manhattan', 'minkowski'],
    'knn__p': [1,5,10,15,20]
}

# Use GridSearchCV here with cv=5
knn_model = GridSearchCV(knn_pl, parameters_knn, cv=5, scoring='f1')

knn_model.fit(X_train, y_train)
print(f'knn_best_parameters : {knn_model.best_params_}') 

# Return best parameters in a dictionary
knn_best_parameters = knn_model.best_params_

step10_data = knn_model, knn_best_parameters, knn_pl

/Users/lizhenkai/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/lizhenkai/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no 

/Users/lizhenkai/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/lizhenkai/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no 

/Users/lizhenkai/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/lizhenkai/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no 

/Users/lizhenkai/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/lizhenkai/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no 

/Users/lizhenkai/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/lizhenkai/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no 

...

/Users/lizhenkai/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/lizhenkai/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no 

/Users/lizhenkai/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/lizhenkai/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no 

/Users/lizhenkai/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/lizhenkai/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no 

knn_best_parameters : {'knn__algorithm': 'ball_tree', 'knn__metric': 'manhattan', 'knn__n_neighbors': 5, 'knn__p': 1, 'knn__weights': 'uniform'}


/Users/lizhenkai/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [18]:
step10_data[1]

{'knn__algorithm': 'ball_tree',
 'knn__metric': 'manhattan',
 'knn__n_neighbors': 5,
 'knn__p': 1,
 'knn__weights': 'uniform'}

**Train and optimise the hyperparameters** to give the best performance for a **Decision tree classifier** classifier. For this model consider and  evaluate the following parameters:

* criterion
* max_depth (with no value greater than 20)
* min_samples_split (with no value greater than 20)
* min_samples_leaf (with no value greater than 20)
* max_features



In [19]:
from sklearn.tree import DecisionTreeClassifier
# Put the pipeline with the appropriate model 
dt_pl = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
    ('classifier', DecisionTreeClassifier())
])

parameters_dt = {
    'classifier__criterion': ['gini', 'entropy'],
    'classifier__max_depth': [1,5,10,15,20],
    'classifier__min_samples_split': [1,5,10,15,20],
    'classifier__min_samples_leaf': [1,5,10,15,20],
    'classifier__max_features': ['auto', 'sqrt', 'log2']
}


# Use GridSearchCV with cv=5
dt_model = GridSearchCV(dt_pl, parameters_dt, cv=5, scoring='f1')

dt_model.fit(X_train, y_train)
# Return best parameters in a dictionary
dt_best_parameters = dt_model.best_params_

step11_data = dt_model,dt_best_parameters, dt_pl

/Users/lizhenkai/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
750 fits failed out of a total of 3750.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
750 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/lizhenkai/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/lizhenkai/opt/anaconda3/lib/python3.9/site-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/lizhenkai/opt/anaconda3/lib/python3.9/site-packages/sklearn/tree/_classes.py", line 937, in 

In [20]:
step11_data[1]

{'classifier__criterion': 'entropy',
 'classifier__max_depth': 10,
 'classifier__max_features': 'sqrt',
 'classifier__min_samples_leaf': 1,
 'classifier__min_samples_split': 10}

**Train and optimise the hyperparameters** to give the best performance for a **C-Support Vector Classification** classifier. For this model consider and  evaluate the following parameters:

* C. Regularization parameter (with no value greater than 100)
* kernel
* gamma


In [21]:
from sklearn.svm import SVC
# Put the pipeline with the appropriate model 
svc_pl = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
    ('classifier', SVC())
])

parameters_svc = {
    'classifier__C': [1,20,40,60,80,100],
    'classifier__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'classifier__gamma': ['scale', 'auto']
}


# Use GridSearchCV with cv=5 
svc_model = GridSearchCV(svc_pl, parameters_svc, cv=5, scoring='f1')

svc_model.fit(X_train, y_train)
# Return best parameters in a dictionary
svc_best_parameters = svc_model.best_params_

step12_data = svc_model,svc_best_parameters, svc_pl

In [22]:
step12_data[1]

{'classifier__C': 1, 'classifier__gamma': 'scale', 'classifier__kernel': 'rbf'}

**Train and optimise the hyperparameters** to give the best performance for a **SGD classifier** classifier. For this model consider and  evaluate the following parameters:

* loss 
* penalty
* alpha (with no value greater than 100)
* learning_rate
* eta0 (with no value greater than 100)


In [23]:
from sklearn.linear_model import SGDClassifier
# Put the pipeline with the appropriate model 
sgd_pl = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
    ('classifier', SGDClassifier())
])

parameters_sgd = {
    'classifier__loss': ['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'],
    'classifier__penalty': ['l1', 'l2', 'elasticnet'],
    'classifier__alpha': [1,20,40,60,80,100],
    'classifier__learning_rate': ['constant', 'optimal', 'invscaling', 'adaptive'],
    'classifier__eta0': [1,20,40,60,80,100]
}
# Use GridSearchCV with cv=5
sgd_model = GridSearchCV(sgd_pl, parameters_sgd, cv=5, scoring='f1')

sgd_model.fit(X_train, y_train)
# Return best parameters in a dictionary
sgd_best_parameters = sgd_model.best_params_

step13_data = sgd_model,sgd_best_parameters, sgd_pl

/Users/lizhenkai/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/Users/lizhenkai/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/Users/lizhenkai/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/Users/lizhenkai/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.wa

In [24]:
step13_data

(GridSearchCV(cv=5,
              estimator=Pipeline(steps=[('imputer',
                                         SimpleImputer(strategy='median')),
                                        ('scaler', StandardScaler()),
                                        ('classifier', SGDClassifier())]),
              param_grid={'classifier__alpha': [1, 20, 40, 60, 80, 100],
                          'classifier__eta0': [1, 20, 40, 60, 80, 100],
                          'classifier__learning_rate': ['constant', 'optimal',
                                                        'invscaling',
                                                        'adaptive'],
                          'classifier__loss': ['hinge', 'log', 'modified_huber',
                                               'squared_hinge', 'perceptron'],
                          'classifier__penalty': ['l1', 'l2', 'elasticnet']},
              scoring='f1'),
 {'classifier__alpha': 1,
  'classifier__eta0': 60,
  'classifier__learning_r

Considering the previous results using **GridSearchCV** for each one of the main classifiers, retrieve the obtained score using the best parameters in each case. Also indicate 'sgd', 'svc', 'knn' are the three **best** models. 

In [25]:
# Assign the best score for each model 
knn_best_cv_scoring = knn_model.best_score_
dt_best_cv_scoring  = dt_model.best_score_
svc_best_cv_scoring = svc_model.best_score_
sgd_best_cv_scoring = sgd_model.best_score_

# print(knn_best_cv_scoring)
# print(dt_best_cv_scoring)
# print(svc_best_cv_scoring)
# print(sgd_best_cv_scoring)

best_models = ["sgd","svc","knn"]

step14_data = best_models,knn_best_cv_scoring,dt_best_cv_scoring,svc_best_cv_scoring,sgd_best_cv_scoring

In [26]:
step14_data

(['sgd', 'svc', 'knn'],
 0.954478260979809,
 0.9437651096474626,
 0.956263756882952,
 0.9653032711856241)

In [27]:
# Function to check the performance of each parameter.
def pooled_var(stds):
    # https://en.wikipedia.org/wiki/Pooled_variance#Pooled_standard_deviation
    n = 5 # size of each group
    return np.sqrt(sum((n-1)*(stds**2))/ len(stds)*(n-1))

# Function to create loss curves
def plot_gridSearchCV_loss_curve(cv_results,grid_params,title):

    df = pd.DataFrame(cv_results)
    results = ['mean_test_score',
               'mean_train_score',
               'std_test_score',
               'std_train_score']


    fig, axes = plt.subplots(1, len(grid_params),
                             figsize = (5*len(grid_params), 7),
                             sharey='row')
    axes[0].set_ylabel("Score", fontsize=25)


    for idx, (param_name, param_range) in enumerate(grid_params.items()):
        grouped_df = df.groupby(f'param_{param_name}')[results]\
            .agg({'mean_train_score': 'mean',
                  'mean_test_score': 'mean',
                  'std_train_score': pooled_var,
                  'std_test_score': pooled_var})

        previous_group = df.groupby(f'param_{param_name}')[results]
        axes[idx].set_xlabel(param_name, fontsize=30)
        axes[idx].set_ylim(0.0, 1.1)
        lw = 2
        axes[idx].plot(param_range, grouped_df['mean_train_score'], label="Training score",
                    color="darkorange", lw=lw)
        axes[idx].fill_between(param_range,grouped_df['mean_train_score'] - grouped_df['std_train_score'],
                        grouped_df['mean_train_score'] + grouped_df['std_train_score'], alpha=0.2,
                        color="darkorange", lw=lw)
        axes[idx].plot(param_range, grouped_df['mean_test_score'], label="Cross-validation score",
                    color="navy", lw=lw)
        axes[idx].fill_between(param_range, grouped_df['mean_test_score'] - grouped_df['std_test_score'],
                        grouped_df['mean_test_score'] + grouped_df['std_test_score'], alpha=0.2,
                        color="navy", lw=lw)

    handles, labels = axes[0].get_legend_handles_labels()
    fig.suptitle(f'{title} Validation curves', fontsize=30)
    fig.legend(handles, labels, loc=8, ncol=2, fontsize=20)

    fig.subplots_adjust(bottom=0.25, top=0.85)
    plt.show()
    
# Check the performance for each model (knn, dt, svc and sgd). Use plot_gridSearchCV_loss_curve() function.




Having identified the best three models in last step, train them using their best parameters on the training data, and then make predictions on the validation set. Finally, calculate and display the following metrics for each case:

* recall
* precision
* auc score
* f1 score

In [28]:

predictions_model1 = sgd_model.predict(X_val)
predictions_model2 = svc_model.predict(X_val)
precictions_model3 = knn_model.predict(X_val)

# calculate_metrics()

metrics_model1 = calculate_metrics(y_val,predictions_model1)
metrics_model2 = calculate_metrics(y_val,predictions_model2)
metrics_model3 = calculate_metrics(y_val,precictions_model3)

step16_data = [(predictions_model1, predictions_model2, precictions_model3),(metrics_model1, metrics_model2, metrics_model3)]

/Users/lizhenkai/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [29]:
step16_data

[(array([1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1,
         0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0,
         1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1]),
  array([1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1,
         0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0,
         1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1]),
  array([1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1,
         0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0,
         1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0])),
 ((0.896551724137931, 1.0, 0.9482758620689655, 0.9454545454545454),
  (0.896551724137931,
   0.9629629629629629,
   0.932146829810901,
   0.9285714285714286),
  (0.896551724137931, 1.0, 0.9482758620689655, 0.9454545454545454))]

**Final performance**

Choose the best model of the top 3 obtained before. Calculate and display an unbiased performance measure that can present to the client.

In [30]:

pipeline_best_sgd = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
    ('classifier', SGDClassifier(alpha = sgd_model.best_params_['classifier__alpha'], eta0 = sgd_model.best_params_['classifier__eta0'], learning_rate = sgd_model.best_params_['classifier__learning_rate'], loss = sgd_model.best_params_['classifier__loss'], penalty = sgd_model.best_params_['classifier__penalty']))
])

X_train_final = np.concatenate((X_train,X_val),axis=0)
y_train_final = np.concatenate((y_train,y_val),axis=0)

# Assign the best model to the variable below and train it
final_model = pipeline_best_sgd.fit(X_train_final,y_train_final)


# Assign the predictions made from my chosen best model for the unbiased estimate to the variable below.
predictions_final_model = final_model.predict(X_test)

# Choose a performance metric based on the client's requirement and assign the result to the variable below.
chosen_performance_metric_result = calculate_metrics(y_test,predictions_final_model)[3]

step17_data = (predictions_final_model, chosen_performance_metric_result, X_train_final, y_train_final,final_model)

/Users/lizhenkai/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SimpleImputer was fitted without feature names
  warnings.warn(


In [31]:
step17_data

(array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0,
        1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1,
        0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1]),
 0.9824561403508771,
 array([[1.55522502e+01, 1.93723534e+01, 1.01558673e+02, ...,
         1.42870889e-01, 2.98967354e-01, 8.42957006e-02],
        [1.38932900e+01, 1.80746025e+01, 9.03211807e+01, ...,
         1.34551096e-01, 3.10931840e-01, 9.00948123e-02],
        [1.27574713e+01, 1.53492704e+01, 8.23755105e+01, ...,
         8.82221181e-02, 2.59796352e-01, 8.12761484e-02],
        ...,
        [1.82002988e+01, 2.13093393e+01, 1.21247839e+02, ...,
         1.79251074e-01, 2.73243777e-01, 9.27802770e-02],
        [1.52934391e+01, 2.03385342e+01, 1.00120886e+02, ...,
         1.26230010e-01, 2.92633539e-01, 8.01402676e-02],
        [1.43879696e+01, 2.07112274e+01, 9.49569823e+01, ...,
         1.35370927e-01, 2.93312800e-01, 8.49036961e-02]]),
 array([1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 

**Rank features**

Although it is only possible to know the true usefulness of a feature when I've combined it with others in a machine learning method, it is still helpful to have some measure for how discriminative each feature is on its own.  One common method for doing this is to calculate a *T-score* (often used in statistics, and in the **LDA** machine learning method) for each feature. The formula for the *T-score* is 

$$
  T_{score} =  \frac{(mean(x_{2}) - mean(x_{1}))}{0.5(stddev(x_{2}) + stddev(x_{1}))}
$$

where $x_{1}$ and $x_{2}$ are the datasets corresponding to the two classes. Large values for the *T-score* (either positive or negative) indicate discriminative ability. Define a function which returns this *T-score*. For this process use the entire dataset.

**Using the defined function, calculate the *T-score* for each feature and obtain the best 4 features according to this score.**

In [32]:
# Implement this function to calculate T score
def calculate_t_score(x1, x2):
    part1 = x2.mean()-x1.mean()
    part2 = 0.5*(x2.std()+x1.std())
    
    return part1/part2

# Split the training data into two datasets corresponding to the two classes
positive = data[data['label'] == 1]
negative = data[data['label'] == 0]
# Calculate the T score for the feature

T_score =calculate_t_score(positive,negative)
# Assign the top 4 features (in order) to this variable. It should be a list containing 4 feature names (strings).
best_four_features = T_score.sort_values(ascending=True).head(5)
best_four_features = ["worst concave points", "worst perimeter", "worst radius","mean concave points"]

step18_data = best_four_features.copy()

In [33]:
# step18_data

In [34]:
# from sklearn.inspection import DecisionBoundaryDisplay

# Assign the name of the best feature obtained in step18 to the variable below. (string)
feature_one = "worst concave points"

# Assign the name of the second best feature obtained in step18 to the variable below. (string)
feature_two = "worst perimeter"

data2d = bigtrain_set[[feature_one, feature_two]]

step19_data = feature_one, feature_two, data2d

In [35]:
step19_data

('worst concave points',
 'worst perimeter',
      worst concave points  worst perimeter
 4                0.131715       124.002529
 248              0.092713        96.343295
 214              0.138745       110.858888
 39               0.153227       105.764034
 6                0.142871       123.466254
 ..                    ...              ...
 14               0.152006       108.230008
 23               0.145247       136.673325
 64               0.137524       109.011659
 109              0.101064        97.638975
 90               0.099083       105.933296
 
 [240 rows x 2 columns])

## Second Round

After presenting my initial results to the client they come back to me and say that they have done some financial analysis and it would save them a lot of time and money if they did not have to analyse every cell, which is needed to get the "worst" features. Instead, they can quickly get accurate estimates for the "mean" and "standard error" features from a much smaller, randomly selected set of cells.

Create a new dataset according to the specifications of the client in round 2. After that, split the data appropriately.

In [36]:
# Assign the data to each variable appropriately

mean_features = [col for col in data.columns if "mean" in col]
se_features = [col for col in data.columns if "error" in col]

subset_size = 100
data_subset = data.sample(n=subset_size, random_state=42)
new_data = data_subset[["label"]+mean_features+se_features]

bigtrain_set, test_set = train_test_split(new_data, test_size=0.2, random_state=42, stratify=new_data.iloc[:,0])
train_set, val_set = train_test_split(bigtrain_set, test_size=0.25, random_state=42, stratify=bigtrain_set.iloc[:,0])

X_train_r2 = train_set.iloc[:,1:]
y_train_r2 = train_set.iloc[:,0]
X_test_r2 = test_set.iloc[:,1:]
y_test_r2 = test_set.iloc[:,0]
X_val_r2 = val_set.iloc[:,1:]
y_val_r2 = val_set.iloc[:,0]

# Print the size of each resulting subset
print("Training set size (second round):", len(X_train_r2))
print("Validation set size (second round):", len(X_val_r2))
print("Test set size (second round):", len(X_test_r2))

step20_data = [X_train_r2.shape, X_val_r2.shape, X_test_r2.shape]

Training set size (second round): 60
Validation set size (second round): 20
Test set size (second round): 20


In [37]:
data_subset

,label,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
203,1,14.008635,21.030820,91.788345,634.049633,0.109099,0.130404,0.112563,0.063592,0.198023,...,17.352981,31.705057,115.186785,983.869900,0.164583,0.351442,0.344269,0.145625,0.343968,0.092702
266,0,12.741679,19.178071,83.014021,530.517644,0.096582,0.107377,0.077936,0.039711,0.184377,...,14.550399,24.708114,95.802414,697.951396,0.128544,0.246237,0.236233,0.099978,0.291153,0.080569
152,0,12.405112,17.920480,80.810155,513.094010,0.099248,0.120756,0.201971,0.057485,0.207309,...,14.216233,23.473571,93.017321,683.061219,0.129877,0.254528,0.460931,0.126505,0.295560,0.098604
9,1,13.561893,20.963861,89.362028,592.703670,0.104794,0.154252,0.139630,0.062670,0.189246,...,15.862556,31.065873,103.973363,817.322205,0.151407,0.543118,0.569733,0.153265,0.341240,0.128288
233,1,16.360102,22.644103,106.826942,885.732764,0.093902,0.105805,0.113261,0.060993,0.169080,...,19.085113,30.299117,126.480988,1217.921903,0.127736,0.262880,0.321713,0.129290,0.272944,0.083251
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16,1,14.063646,19.594314,91.224735,646.433727,0.097583,0.091010,0.080046,0.048436,0.173515,...,16.888596,27.606738,110.334688,936.409712,0.137263,0.220312,0.265185,0.125832,0.290965,0.083345
179,0,13.633879,17.007978,87.984797,601.826423,0.092679,0.079717,0.058404,0.035539,0.168299,...,15.293258,22.150202,99.684395,768.857044,0.126318,0.183191,0.178870,0.086184,0.248362,0.080451
147,0,14.292182,19.172632,93.208418,652.350208,0.091921,0.108607,0.090066,0.044576,0.179233,...,16.102088,25.579282,106.052623,835.639424,0.120934,0.250183,0.261961,0.103492,0.288199,0.086631
274,1,15.566380,21.256851,100.781334,784.702608,0.093415,0.092329,0.077884,0.048925,0.171039,...,18.002922,29.123553,117.729587,1045.765228,0.132138,0.228518,0.254089,0.115998,0.275859,0.082443


Train the best model obtained in the first round on the new dataset and find the best parameters.

In [38]:
# sgd_pl = Pipeline([
#     ('imputer', SimpleImputer(strategy='median')),
#     ('scaler', StandardScaler()),
#     ('classifier', SGDClassifier())
# ])

# Assign the GridSearchCV instance have obtained in this step to the variable below.
final_model_r2 = GridSearchCV(sgd_pl, parameters_sgd, cv=5, scoring='f1')
final_model_r2.fit(X_train_r2, y_train_r2)
print(f'best_parameters_r2: {final_model_r2.best_params_}')

# Assign the best parameter dictionary to the variable below.
best_parameters_r2 = final_model_r2.best_params_

step21_data = final_model_r2, best_parameters_r2

/Users/lizhenkai/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/Users/lizhenkai/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/Users/lizhenkai/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/Users/lizhenkai/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.wa

best_parameters_r2: {'classifier__alpha': 80, 'classifier__eta0': 100, 'classifier__learning_rate': 'adaptive', 'classifier__loss': 'perceptron', 'classifier__penalty': 'l2'}


In [39]:
step21_data

(GridSearchCV(cv=5,
              estimator=Pipeline(steps=[('imputer',
                                         SimpleImputer(strategy='median')),
                                        ('scaler', StandardScaler()),
                                        ('classifier', SGDClassifier())]),
              param_grid={'classifier__alpha': [1, 20, 40, 60, 80, 100],
                          'classifier__eta0': [1, 20, 40, 60, 80, 100],
                          'classifier__learning_rate': ['constant', 'optimal',
                                                        'invscaling',
                                                        'adaptive'],
                          'classifier__loss': ['hinge', 'log', 'modified_huber',
                                               'squared_hinge', 'perceptron'],
                          'classifier__penalty': ['l1', 'l2', 'elasticnet']},
              scoring='f1'),
 {'classifier__alpha': 80,
  'classifier__eta0': 100,
  'classifier__learning

Considering the best parameters identified in the previous step, train the best model using those parameters and calculate and display an unbiased performance measure.

In [40]:
pipeline_best_sgd_r2 = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
    ('classifier', SGDClassifier(alpha = final_model_r2.best_params_['classifier__alpha'], eta0 = final_model_r2.best_params_['classifier__eta0'], learning_rate = final_model_r2.best_params_['classifier__learning_rate'], loss = final_model_r2.best_params_['classifier__loss'], penalty = final_model_r2.best_params_['classifier__penalty']))
])

X_train_final_r2 = np.concatenate((X_train_r2,X_val_r2),axis=0)
y_train_final_r2 = np.concatenate((y_train_r2,y_val_r2),axis=0)

# Assign the best model I have trained again in order to get an unbiased performance estimate.
final_model_r2 = pipeline_best_sgd_r2.fit(X_train_final_r2,y_train_final_r2)

# Assign the predictions made from my chosen best model for the unbiased estimate to the variable below.
predictions_final_model_r2 = final_model_r2.predict(X_test_r2)

# Choose a performance metric based on the client's requirement and assign the result to the variable below.
performance_metric_result_r2 = calculate_metrics(y_test_r2,predictions_final_model_r2)[3]

step22_data = (predictions_final_model_r2, performance_metric_result_r2,X_train_final_r2, y_train_final_r2,final_model_r2)

/Users/lizhenkai/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SimpleImputer was fitted without feature names
  warnings.warn(


In [41]:
step22_data

(array([1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1]),
 0.8571428571428572,
 array([[1.27013821e+01, 1.76439722e+01, 8.20161755e+01, ...,
         9.58732457e-03, 1.92578148e-02, 3.31825354e-03],
        [1.29563811e+01, 1.76187157e+01, 8.38590744e+01, ...,
         1.12597412e-02, 1.93379063e-02, 3.44900330e-03],
        [1.34465273e+01, 1.79572983e+01, 8.81108998e+01, ...,
         1.15694764e-02, 2.05737209e-02, 4.21423163e-03],
        ...,
        [1.26341466e+01, 1.88430965e+01, 8.13406646e+01, ...,
         7.65134815e-03, 2.67620598e-02, 2.91398256e-03],
        [1.43879696e+01, 2.07112274e+01, 9.49569823e+01, ...,
         1.44179562e-02, 2.22314673e-02, 3.93933037e-03],
        [1.22792827e+01, 1.79735609e+01, 7.97009479e+01, ...,
         1.16563613e-02, 1.97554970e-02, 4.03949518e-03]]),
 array([0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1,
        0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0,
        1, 0, 1, 0, 0, 1, 

Compare the final performance obtained in round 1 with the performance given in round 2.

In [42]:
# Final performance for 2nd round
final_performance_r1 = chosen_performance_metric_result

# Final performance for 2nd round
final_performance_r2 = performance_metric_result_r2

final_answer = "r1"

step23_data = (final_performance_r1, final_performance_r2,final_answer)

In [43]:
step23_data

(0.9824561403508771, 0.8571428571428572, 'r1')

The client has provided data about a new patient in a table format, and is expecting me to make a prediction about the person's condition as soon as possible. The following table contains the information provided by the client.

| Characteristic | Value |
| --- | --- |
| mean radius |13.970546 | 
| mean texture|15.660529 | 
| mean perimeter | 91.432976| 
| mean area |600.575775 | 
| mean smoothness |0.099883 | 
| mean compactness | NaN| 
| mean concavity | 0.07505| 
| mean concave points | 0.023223| 
| mean symmetry | 0.186472| 
| mean fractal dimension | 0.05446|
| radius error | 0.321022 | 
| texture error |1.044633 | 
| perimeter error |2.324773 | 
| area error |31.333479 | 
| smoothness error | 0.005675| 
| compactness error |0.023401 | 
| concavity error |0.026742 | 
| concave points error |0.009808 | 
| symmetry error | 0.029254| 
| fractal dimension error |0.00325 |
| worst radius |12.728135 | 
| worst texture | 23.476671| 
| worst perimeter |103.834522 | 
| worst area | 798.832368| 
| worst smoothness |0.130751 | 
| worst compactness |0.264469 | 
| worst concavity |0.247665 | 
| worst concave points | 0.100153| 
| worst symmetry |0.301228 | 
| worst fractal dimension |0.083051 |  

My goal is to inform the client whether the sample corresponds to a benign or malignant case. Provide the class and also the probability that the sample belongs to the malignant class.

In [50]:
# Put the new data in form of a dataframe

data = {'mean radius': [13.970546], 'mean texture': [15.660529], 'mean perimeter': [91.432976],
        'mean area': [600.575775], 'mean smoothness': [0.099883], 'mean compactness': [pd.np.nan],
        'mean concavity': [0.07505], 'mean concave points': [0.023223], 'mean symmetry': [0.186472],
        'mean fractal dimension': [0.05446], 'radius error': [0.321022], 'texture error': [1.044633],
        'perimeter error': [2.324773], 'area error': [31.333479], 'smoothness error': [0.005675],
        'compactness error': [0.023401], 'concavity error': [0.026742], 'concave points error': [0.009808],
        'symmetry error': [0.029254], 'fractal dimension error': [0.00325], 'worst radius': [12.728135],
        'worst texture': [23.476671], 'worst perimeter': [103.834522], 'worst area': [798.832368],
        'worst smoothness': [0.130751], 'worst compactness': [0.264469], 'worst concavity': [0.247665],
        'worst concave points': [0.100153], 'worst symmetry': [0.301228], 'worst fractal dimension': [0.083051]}
new_sample = pd.DataFrame(data)
# Print the data
print(new_sample)

# Use the best model to predict. 
predict = final_model.predict(new_sample)

prediction_sample = "benign"

# Probability or confidence score of the sample belonging to the malignant class
# probability_sample = ...

print("The class for this sample is ", prediction_sample)
probability_sample = final_model.decision_function(new_sample)

# NOTE: in the case of SGD, Confidence score values will not be bounded within 0 and 1.
print("The probability / confidence score of belonging to malignant class is ", probability_sample)

step24_data = (new_sample,prediction_sample, probability_sample)

   mean radius  mean texture  mean perimeter   mean area  mean smoothness  \
0    13.970546     15.660529       91.432976  600.575775         0.099883   

   mean compactness  mean concavity  mean concave points  mean symmetry  \
0               NaN         0.07505             0.023223       0.186472   

   mean fractal dimension  ...  worst radius  worst texture  worst perimeter  \
0                 0.05446  ...     12.728135      23.476671       103.834522   

   worst area  worst smoothness  worst compactness  worst concavity  \
0  798.832368          0.130751           0.264469         0.247665   

   worst concave points  worst symmetry  worst fractal dimension  
0              0.100153        0.301228                 0.083051  

[1 rows x 30 columns]
The class for this sample is  benign
The probability / confidence score of belonging to malignant class is  [-0.95089852]


/var/folders/ql/s8bjk9n15n5818m7wqz08tj80000gn/T/ipykernel_15817/1910646759.py:4: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  'mean area': [600.575775], 'mean smoothness': [0.099883], 'mean compactness': [pd.np.nan],
/Users/lizhenkai/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SimpleImputer was fitted without feature names
  warnings.warn(
/Users/lizhenkai/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SimpleImputer was fitted without feature names
  warnings.warn(
